<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/104_TxtSummarizerAgent_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here’s a quick audit of your agent against the **Agent Builder Handbook** and exactly what (if anything) to add.

## What you already match

* **GAME separation** — Goals (in memory), Actions (tools + registry), Memory (ScratchMemory), Environment (exec + DI + logging). ✅&#x20;
* **Tool naming + single-job tools** — clear verbs; each tool does one thing. ✅&#x20;
* **Central orchestrator** — your `ScriptedAgent + Environment` fills the “Orchestrator loop” role for a fixed pipeline. ✅&#x20;
* **Schemas** — parameters documented in the registry; pre-call validation in `Environment.run`. ✅&#x20;
* **Determinism & truncation notes** — prompt truncation recorded; consistent progress logging. ✅&#x20;

## Gaps vs. the handbook (with minimal, surgical patches)

1. **Standard return envelope (`ok` / `error` / `hint` / `retryable`)**
   *Why:* Handbook recommends a consistent shape for success and errors; makes logs, testing, and planner integration cleaner.&#x20;
   **Patch (helpers):**

   ```python
   def ok(**data): return {"ok": True, **data}
   def err(msg, hint=None, retryable=False):
       d = {"ok": False, "error": msg, "retryable": retryable}
       if hint: d["hint"] = hint
       return d
   ```

   **Use in tools (example):**

   ```python
   if not os.path.exists(path):
       return err(f"File not found: {path}",
                  hint="Confirm the name or call list_txt_files", retryable=True)
   return ok(message="File read successfully.", length=len(text))
   ```

   **Tiny env tweak:** treat `{"ok": False}` as error same as `{"error": ...}`.&#x20;

2. **JIT error guidance (“next best step”)**
   *Why:* The handbook’s “coach” pattern—errors should guide the agent on recovery (e.g., suggest `list_txt_files`).&#x20;
   **Patch:** add `hint` (as above) and consider a tiny `list_txt_files` tool:

   ```python
   def list_txt_files(ctx):
       base = ctx.config["input_folder"]
       names = [f for f in os.listdir(base) if f.endswith(".txt")]
       return ok(files=sorted(names))
   ```

3. **Path-safety + size guards (Environment checklist)**
   *Why:* Prevent path traversal and unexpected huge reads.&#x20;
   **Patch (in `read_txt_file`):**

   ```python
   base = os.path.abspath(ctx.config["input_folder"])
   path = os.path.abspath(os.path.join(base, file_name))
   if not path.startswith(base + os.sep):
       return err("Path traversal blocked.", retryable=False)
   # optional: cap size before read if needed
   ```

4. **In-memory or fake environment for tests**
   *Why:* Handbook recommends a simulated env to run dress rehearsals with no side effects. You’re close already thanks to DI; just show one injected dep.&#x20;
   **Patch (underscore-dep example):**

   ```python
   def save_summary(ctx, out_name=None, _fs=os):
       # use _fs.path.join, _fs.makedirs, _fs.open(...)
   # test: ctx.deps["fs"] = FakeFS()  # drop-in fake
   ```

5. **(Optional) “AgentLanguage”/Orchestrator format**
   *Why:* If you later want the LLM to choose tools, the handbook uses a small “AgentLanguage” layer plus the orchestrator loop. Your `ScriptedAgent` is fine now; consider this only when you switch from fixed steps to model-driven steps.&#x20;

6. **Top-10 rules / quick reference**
   *Why:* Handbook suggests keeping a concise rules section for future you. Add it to your notes; no code changes.&#x20;




In [ ]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap
import time
import re
import inspect
from typing import Callable, Optional
from dataclasses import dataclass
import builtins

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)

# ---------- Standard result envelope ----------
def ok(**data):
    """Successful tool result. Add any fields you like."""
    return {"ok": True, **data}

def err(msg, hint=None, retryable=False, **extra):
    """Error result with optional guidance and flags."""
    out = {"ok": False, "error": msg, "retryable": retryable}
    if hint:
        out["hint"] = hint
    if extra:
        out.update(extra)
    return out


# FS adapter
class RealFS:
    path = os.path
    makedirs = staticmethod(os.makedirs)
    open = staticmethod(builtins.open)

# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key, default=None):   # <-- add default
        return self.store.get(key, default)

    def set(self, key, value):
        self.store[key] = value

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None, deps=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}
        self.deps = deps or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None

# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini", temperature=0.2):
        self.client = client
        self.model = model
        self.temperature = temperature

    def complete(self, prompt, **kwargs):
        temp = kwargs.get("temperature", self.temperature)
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
        )
        return resp.choices[0].message.content

# ---------------- Tool: create_plan ----------------
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return err("No goal provided (memory key 'goal' missing).",
                   hint="Set ctx.memory['goal'] before calling create_plan")

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond ONLY with a numbered list, one step per line. No extra prose."""
    raw = ctx.llm.complete(prompt).strip()

    # Prefer numbered steps like "1. ...", "2) ..."
    numbered = re.findall(r'^\s*(?:\d+[\).\s-]+)\s*(.+)$', raw, flags=re.M)

    if numbered:
        steps = numbered
    else:
        # Fallback to bullets like "- ...", "* ...", "• ..."
        bullets = re.findall(r'^\s*(?:[-*•]\s+)(.+)$', raw, flags=re.M)
        steps = bullets if bullets else [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # Normalize: collapse spaces, trim punctuation, drop empties/dupes
    clean_steps = []
    seen = set()
    for s in steps:
        s = re.sub(r'\s+', ' ', s).strip(' .')
        if s and s.lower() not in seen:
            seen.add(s.lower())
            clean_steps.append(s)

    if not clean_steps:
        return err("Planner returned no steps.",
                   hint="Refine the goal or relax the parser constraints")

    ctx.memory.set("plan", clean_steps)
    # optional: match handbook wording
    # ctx.memory.set("current_plan", clean_steps)

    return ok(message="Plan created from goal.", steps=clean_steps)


# ---------------- Tool: read_txt_file ----------------
def read_txt_file(ctx, file_name):
    base = os.path.abspath(ctx.config.get("input_folder", ""))
    path = os.path.abspath(os.path.join(base, file_name))
    if not base or not path.startswith(base + os.sep):
        return err("Path traversal blocked.", retryable=False)

    if not os.path.exists(path):
        return err(f"File not found: {path}",
                   hint="Call list_txt_files to see available files",
                   retryable=True)

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("file_name", file_name)
    ctx.memory.set("raw_text", text)
    return ok(message="File read successfully.", length=len(text))

# --------------------Tool: list_text_file -----------------
def list_txt_files(ctx):
    base = ctx.config.get("input_folder")
    if not base:
        return err("No input_folder in config.", hint="Set ctx.config['input_folder']")
    if not os.path.isdir(base):
        return err(f"Input folder not found: {base}", retryable=False)

    files = sorted(f for f in os.listdir(base) if f.endswith(".txt"))
    ctx.memory.set("available_txt_files", files)  # optional: stash for UI/agent
    return ok(message=f"Found {len(files)} .txt files.", files=files, count=len(files))


# ---------------- Tool: generate_summary_prompt ----------------
def generate_summary_prompt(ctx, max_len=None):
    text = ctx.memory.get("raw_text")
    if not text:
        return err("No raw text found in memory.",
                   hint="Run read_txt_file before generate_summary_prompt")
    if max_len is None:
        max_len = ctx.config.get("summary_max_chars", 2000)

    truncated = len(text) > max_len
    short_text = text[:max_len]

    ctx.memory.set("was_truncated", truncated)
    ctx.memory.set("source_length", len(text))
    ctx.memory.set("used_length", len(short_text))
    ctx.memory.set("summary_prompt", f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points...
\"\"\"{short_text}\"\"\"

Summary:""")

    return ok(message="Summary prompt created.",
              truncated=truncated, used=len(short_text), total=len(text),
              prompt_preview=ctx.memory.get("summary_prompt")[:600])


# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return err("No summary prompt found in memory.",
                   hint="Run generate_summary_prompt before summarize")
    response = ctx.llm.complete(prompt)
    ctx.memory.set("summary", response)
    return ok(message="Summary completed.", summary_preview=response[:1000])

# ---------------- Tool: save_summary ----------------
def save_summary(ctx, out_name=None, _fs=os):
    summary = ctx.memory.get("summary")
    if not summary:
        return err("No summary in memory.",
                   hint="Run summarize before save_summary")
    out_dir = ctx.config.get("output_folder")
    if not out_dir:
        return err("No output_folder in config.",
                   hint="Set ctx.config['output_folder']")

    _fs.makedirs(out_dir, exist_ok=True)
    src = ctx.memory.get("file_name", "summary")
    root, _ = os.path.splitext(os.path.basename(src))
    base = out_name or f"{root}_summary.txt"
    path = _fs.path.join(out_dir, base)

    with _fs.open(path, "w", encoding="utf-8") as f:
        f.write(summary)

    ctx.memory.set("summary_path", path)
    return ok(message="Summary saved.", path=path)

# -----------------Tool Registry ----------------
@dataclass
class ToolDef:
    name: str
    func: Callable
    description: str = ""
    schema: dict | None = None
    returns: dict | None = None   # <-- add this line

class ToolRegistry:
    def __init__(self):
        self._tools = {}

    def register(self, tool: ToolDef):
        self._tools[tool.name] = tool

    def get(self, name: str) -> ToolDef:
        if name not in self._tools:
            raise KeyError(f"Unknown tool: {name}")
        return self._tools[name]

    def list(self):
        return list(self._tools.keys())

# ------------build registry------------
registry = ToolRegistry()

registry.register(ToolDef(
    "create_plan",
    create_plan,
    "Create a plan from goal",
    schema={ "type": "object", "properties": {}, "required": [] },   # no kwargs
    returns={
        "type": "object",
        "properties": {
            "message": { "type": "string" },
            "steps":   { "type": "array", "items": { "type": "string" } }
        },
        "required": ["message", "steps"]
    }
))

registry.register(ToolDef(
  "read_txt_file", read_txt_file, "Read a .txt file from input_folder",
  schema={
    "type": "object",
    "properties": {"file_name": {"type": "string"}},
    "required": ["file_name"]
  },
  returns={
    "type": "object",
    "properties": {"message": {"type": "string"}, "length": {"type": "integer"}},
    "required": ["message"]
  },
  # side_effects=("read",), version="1.0"
))

registry.register(ToolDef(
    "list_txt_files",
    list_txt_files,
    "List .txt files in input_folder",
    schema={ "type": "object", "properties": {}, "required": [] },
    returns={
        "type": "object",
        "properties": {
            "ok":    { "type": "boolean" },
            "message": { "type": "string" },
            "files": { "type": "array", "items": { "type": "string" } },
            "count": { "type": "integer" }
        },
        "required": ["ok", "files"]
    }
))

registry.register(ToolDef(
  "generate_summary_prompt", generate_summary_prompt, "Build a summarization prompt",
  schema={
    "type": "object",
    "properties": {"max_len": {"type": "integer", "minimum": 1}},
    "required": []
  }
))

registry.register(ToolDef(
  "summarize", summarize, "Run LLM summarization",
  schema={"type": "object", "properties": {}, "required": []}
))

registry.register(ToolDef(
  "save_summary", save_summary, "Persist summary to output_folder",
  schema={
    "type": "object",
    "properties": {"out_name": {"type": "string"}},
    "required": []
  },
  # side_effects=("write",)
))

#------------Environment--------------
def _validate(schema, kwargs):
    if not schema:
        return None
    missing = [k for k in schema.get("required", []) if k not in kwargs]
    if missing:
        return f"Missing required: {missing}"
    types = {"string": str, "integer": int, "number": (int, float), "boolean": bool}
    for k, spec in (schema.get("properties") or {}).items():
        if k in kwargs and "type" in spec:
            py_t = types.get(spec["type"])
            if py_t and not isinstance(kwargs[k], py_t):
                return f"Bad type for '{k}': expected {spec['type']}"
    return None

class Environment:
    def __init__(self, ctx: ActionContext, registry: ToolRegistry):
        self.ctx = ctx
        self.registry = registry

    def run(self, tool_name: str, **kwargs):
        tool = self.registry.get(tool_name)
        fn = tool.func
        sig = inspect.signature(fn)

        # 1) Schema validation BEFORE logging/exec
        v_err = _validate(tool.schema, kwargs)
        if v_err:
            self.ctx.track_progress(tool.name, "error", note=v_err[:180])
            return err(v_err)  # use standardized envelope

        # 2) Build call args with auto-DI (ctx + underscore deps)
        call_args = {}
        for pname, param in sig.parameters.items():
            if pname == "ctx":
                call_args["ctx"] = self.ctx
            elif pname.startswith("_"):   # underscore dep, e.g. _fs, _clock
                dname = pname[1:]
                if dname not in self.ctx.deps:
                    msg = f"Missing dep '{dname}' for tool '{tool_name}'"
                    self.ctx.track_progress(tool.name, "error", note=msg[:180])
                    return err(msg)
                call_args[pname] = self.ctx.deps[dname]
            else:
                if pname in kwargs:
                    call_args[pname] = kwargs[pname]
                elif param.default is not inspect._empty:
                    pass
                else:
                    msg = f"Missing required arg '{pname}' for tool '{tool_name}'"
                    self.ctx.track_progress(tool.name, "error", note=msg[:180])
                    return err(msg)

        # 3) Log start, call tool
        self.ctx.track_progress(tool.name, "started", note=str(kwargs))
        try:
            result = fn(**call_args)
        except Exception as e:
            # Normalize exceptions into err(...) so the agent can handle them
            msg = f"{type(e).__name__}: {e}"
            self.ctx.track_progress(tool.name, "error", note=msg[:180])
            return err(msg)

        # 4) Normalize + log outcome
        if isinstance(result, dict):
            # If tool used envelope:
            if result.get("ok") is False:
                self.ctx.track_progress(tool.name, "error", note=str(result.get("error", ""))[:180])
                return result
            # Back-compat: dict returned with "error" but no "ok"
            if "ok" not in result and "error" in result:
                self.ctx.track_progress(tool.name, "error", note=str(result["error"])[:180])
                return {"ok": False, **result}
            # Success path: ensure ok=True for consistency
            result = result if "ok" in result else {"ok": True, **result}
            note = result.get("message", "")[:120]
            self.ctx.track_progress(tool.name, "completed", note=note)
            return result

        # Non-dict success (rare): mark completed with empty note
        self.ctx.track_progress(tool.name, "completed", note="")
        return result


# ---------- Scripted Agent ----------
class ScriptedAgent:
    def __init__(self, env, steps):
        self.env = env
        self.steps = steps

    def run(self, max_calls=None, stop_on_error=True):
        calls = 0
        for name, kwargs in self.steps:
            if max_calls is not None and calls >= max_calls:
                return {"final": f"stopped: max_calls={max_calls}"}
            res = self.env.run(name, **(kwargs or {}))
            calls += 1
            if stop_on_error and isinstance(res, dict) and res.get("ok") is False:
                # include hint so you know the next best step
                out = {"final": f"stopped at {name}: {res['error']}"}
                if "hint" in res: out["hint"] = res["hint"]
                return out
        return {"final": "done"}

# ---------------- Setup ----------------
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")

config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output",
    # "summary_max_chars": 2400,  # optional
}

llm = OpenAILLM(
    client,
    model=config.get("model", "gpt-4o-mini"),
    temperature=config.get("temperature", 0.2),
)

# Create context with DI bag pre-populated
ctx = ActionContext(memory=memory, llm=llm, config=config, deps={"fs": RealFS})

# Ensure folders exist
os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

# ------------Build env ------------#
env = Environment(ctx, registry)

# --- define the steps your agent should run ---
file_name = "004_AGENT_Tools.txt"
steps = [
    ("create_plan", {}),
    ("read_txt_file", {"file_name": file_name}),
    ("generate_summary_prompt", {}),  # or {"max_len": 2400}
    ("summarize", {}),
    ("save_summary", {}),
]

# --- run the agent ---
agent = ScriptedAgent(env, steps)
final = agent.run(max_calls=10)  # optional guard
print("Agent result:", final["final"])
if "hint" in final:
    print("💡 Hint:", final["hint"])

# --- pretty prints from memory (no second calls) ---
plan = ctx.memory.get("plan") or []
print("\nPlan:")
for s in plan:
    print("-", s)

raw_text = ctx.memory.get("raw_text") or ""
print("\n📄 File Preview:\n")
print(textwrap.fill(raw_text[:600], width=80, subsequent_indent="  "))

prompt = ctx.memory.get("summary_prompt") or ""
print("\n🧾 Prompt Preview:\n")
print(textwrap.fill(prompt[:600], width=80, subsequent_indent="  "))

summary = ctx.memory.get("summary") or ""
print("\n📝 Summary Preview:\n")
print(textwrap.fill(summary[:1000], width=80, subsequent_indent="  "))

if ctx.memory.get("summary_path"):
    print("\n📄 Saved to:", ctx.memory.get("summary_path"))

print("\n" + "="*80)
print("📦 ActionContext Snapshot")
ctx.print_progress()



Agent result: done

Plan:
- Open the text file
- Read the content of the file
- Identify the main ideas and key points
- Take notes on important details and supporting information
- Organize the notes into a coherent structure
- Write a summary based on the organized notes
- Review and edit the summary for clarity and conciseness

📄 File Preview:

 #===========AI-Agent Tool Descriptions and Naming  Describing Tools to the
  Agent  When developing an agentic AI system, one of the most critical aspects
  is ensuring that the agent understands the tools it has access to. In our
  previous tutorial, we explored how an AI agent interacts with an environment.
  Now, we extend that discussion to focus on tool definition, particularly the
  importance of naming, parameters, and structured metadata.  Example:
  Automating Documentation for Python Code Imagine we are building an AI agent
  that scans through all Python files in a src/ directory and automatical

🧾 Prompt Preview:

You are an expe

We **standardized tool results**. Here’s what changed and why it helps.

## What changed

**Before:** tools returned ad-hoc dicts

```python
return {"message": "Summary saved.", "path": path}
return {"error": "No summary in memory."}
```

**Now:** tools return a **standard envelope**

```python
def ok(**data): return {"ok": True, **data}
def err(msg, hint=None, retryable=False, **extra):
    out = {"ok": False, "error": msg, "retryable": retryable}
    if hint: out["hint"] = hint
    if extra: out.update(extra)
    return out

# in tools
return ok(message="Summary saved.", path=path)
return err("No summary in memory.", hint="Run summarize before save_summary")
```

And the **Environment** understands that envelope:

* validates inputs *before* starting
* logs `error` if `ok=False` (or a legacy `{"error": ...}` dict)
* logs `completed` only when `ok=True`
* converts exceptions into `err(...)` so the agent can stop cleanly

## Why we made the change (benefits)

1. **One way to branch on results**
   Callers can always do:

   ```python
   res = env.run("some_tool", ...)
   if not res.get("ok"):   # error path
       print("❌", res["error"]); print("💡", res.get("hint",""))
   else:                   # success path
       print("✅", res.get("message",""))
   ```

2. **Cleaner, reliable logging**
   The env can mark outcomes correctly because it sees `ok` vs `error` consistently.

3. **Easier orchestration & guards**
   Your `ScriptedAgent` just checks `ok`; on failure it stops (or could retry if `retryable=True`).

4. **Better error UX**
   `hint` tells the user/agent how to recover (e.g., “Run `read_txt_file` first” or “Try `list_txt_files`”).

5. **Future-proof for planners & UIs**
   A planner (LLM or program) can rely on a uniform result shape. UIs/CLIs can render “ok/error/hint” the same way for every tool.

6. **Telemetry & testing**
   Metrics/analytics don’t need tool-specific parsing. In tests you can assert `res["ok"] is False` and check `retryable`.

## Tiny rules for writing tools now

* **Always** return `ok(...)` on success, `err(...)` on failure.
* Put human-readable info in `message` (success) or `error` + optional `hint` (failure).
* Use `retryable=True` when it makes sense (e.g., missing file that might appear later).
* Keep side effects + state updates (e.g., `ctx.memory.set(...)`) the same as before.

That’s the whole idea: one predictable envelope → simpler code, better logs, easier recovery, and ready for more sophisticated agents later.

---

A single, consistent result shape (“`ok` or not”) **shrinks the model’s cognitive load** so it can think about the task—not about your return formats.

### why this helps the LLM/agent

* **Binary outcome = simpler control flow.** No guessing from varied messages; just:

  ```python
  res = env.run("read_txt_file", file_name=...)
  if not res["ok"]:
      # decide: retry? different tool? stop?
  ```
* **Predictable fields.** The model (or your agent loop) always knows where to look: `ok`, `error`, `hint`, `retryable`, `message`.
* **Cleaner recovery.** `hint` tells the next best step (e.g., “call `list_txt_files`”), so fewer tokens spent inferring what to try.
* **Easier policies.** Your loop can apply uniform rules (e.g., retry only when `retryable=True`, hard-stop otherwise).
* **Better logging & debugging.** The env marks `completed` only on `ok=True`, so the trace is unambiguous.

Net effect: **less mental overhead → more thinking budget for planning and summarizing**, which is exactly the best-practice you’ve been following.



## Sanity Check: Incorrect Filepath

In [ ]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap
import time
import re
import inspect
from typing import Callable, Optional
from dataclasses import dataclass
import builtins

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)

# ---------- Standard result envelope ----------
def ok(**data):
    """Successful tool result. Add any fields you like."""
    return {"ok": True, **data}

def err(msg, hint=None, retryable=False, **extra):
    """Error result with optional guidance and flags."""
    out = {"ok": False, "error": msg, "retryable": retryable}
    if hint:
        out["hint"] = hint
    if extra:
        out.update(extra)
    return out


# FS adapter
class RealFS:
    path = os.path
    makedirs = staticmethod(os.makedirs)
    open = staticmethod(builtins.open)

# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key, default=None):   # <-- add default
        return self.store.get(key, default)

    def set(self, key, value):
        self.store[key] = value

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None, deps=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}
        self.deps = deps or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None

# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini", temperature=0.2):
        self.client = client
        self.model = model
        self.temperature = temperature

    def complete(self, prompt, **kwargs):
        temp = kwargs.get("temperature", self.temperature)
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
        )
        return resp.choices[0].message.content

# ---------------- Tool: create_plan ----------------
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return err("No goal provided (memory key 'goal' missing).",
                   hint="Set ctx.memory['goal'] before calling create_plan")

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond ONLY with a numbered list, one step per line. No extra prose."""
    raw = ctx.llm.complete(prompt).strip()

    # Prefer numbered steps like "1. ...", "2) ..."
    numbered = re.findall(r'^\s*(?:\d+[\).\s-]+)\s*(.+)$', raw, flags=re.M)

    if numbered:
        steps = numbered
    else:
        # Fallback to bullets like "- ...", "* ...", "• ..."
        bullets = re.findall(r'^\s*(?:[-*•]\s+)(.+)$', raw, flags=re.M)
        steps = bullets if bullets else [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # Normalize: collapse spaces, trim punctuation, drop empties/dupes
    clean_steps = []
    seen = set()
    for s in steps:
        s = re.sub(r'\s+', ' ', s).strip(' .')
        if s and s.lower() not in seen:
            seen.add(s.lower())
            clean_steps.append(s)

    if not clean_steps:
        return err("Planner returned no steps.",
                   hint="Refine the goal or relax the parser constraints")

    ctx.memory.set("plan", clean_steps)
    # optional: match handbook wording
    # ctx.memory.set("current_plan", clean_steps)

    return ok(message="Plan created from goal.", steps=clean_steps)


# ---------------- Tool: read_txt_file ----------------
def read_txt_file(ctx, file_name):
    base = os.path.abspath(ctx.config.get("input_folder", ""))
    path = os.path.abspath(os.path.join(base, file_name))
    if not base or not path.startswith(base + os.sep):
        return err("Path traversal blocked.", retryable=False)

    if not os.path.exists(path):
        return err(f"File not found: {path}",
                   hint="Call list_txt_files to see available files",
                   retryable=True)

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("file_name", file_name)
    ctx.memory.set("raw_text", text)
    return ok(message="File read successfully.", length=len(text))

# --------------------Tool: list_text_file -----------------
def list_txt_files(ctx):
    base = ctx.config.get("input_folder")
    if not base:
        return err("No input_folder in config.", hint="Set ctx.config['input_folder']")
    if not os.path.isdir(base):
        return err(f"Input folder not found: {base}", retryable=False)

    files = sorted(f for f in os.listdir(base) if f.endswith(".txt"))
    ctx.memory.set("available_txt_files", files)  # optional: stash for UI/agent
    return ok(message=f"Found {len(files)} .txt files.", files=files, count=len(files))


# ---------------- Tool: generate_summary_prompt ----------------
def generate_summary_prompt(ctx, max_len=None):
    text = ctx.memory.get("raw_text")
    if not text:
        return err("No raw text found in memory.",
                   hint="Run read_txt_file before generate_summary_prompt")
    if max_len is None:
        max_len = ctx.config.get("summary_max_chars", 2000)

    truncated = len(text) > max_len
    short_text = text[:max_len]

    ctx.memory.set("was_truncated", truncated)
    ctx.memory.set("source_length", len(text))
    ctx.memory.set("used_length", len(short_text))
    ctx.memory.set("summary_prompt", f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points...
\"\"\"{short_text}\"\"\"

Summary:""")

    return ok(message="Summary prompt created.",
              truncated=truncated, used=len(short_text), total=len(text),
              prompt_preview=ctx.memory.get("summary_prompt")[:600])


# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return err("No summary prompt found in memory.",
                   hint="Run generate_summary_prompt before summarize")
    response = ctx.llm.complete(prompt)
    ctx.memory.set("summary", response)
    return ok(message="Summary completed.", summary_preview=response[:1000])

# ---------------- Tool: save_summary ----------------
def save_summary(ctx, out_name=None, _fs=os):
    summary = ctx.memory.get("summary")
    if not summary:
        return err("No summary in memory.",
                   hint="Run summarize before save_summary")
    out_dir = ctx.config.get("output_folder")
    if not out_dir:
        return err("No output_folder in config.",
                   hint="Set ctx.config['output_folder']")

    _fs.makedirs(out_dir, exist_ok=True)
    src = ctx.memory.get("file_name", "summary")
    root, _ = os.path.splitext(os.path.basename(src))
    base = out_name or f"{root}_summary.txt"
    path = _fs.path.join(out_dir, base)

    with _fs.open(path, "w", encoding="utf-8") as f:
        f.write(summary)

    ctx.memory.set("summary_path", path)
    return ok(message="Summary saved.", path=path)

# -----------------Tool Registry ----------------
@dataclass
class ToolDef:
    name: str
    func: Callable
    description: str = ""
    schema: dict | None = None
    returns: dict | None = None   # <-- add this line

class ToolRegistry:
    def __init__(self):
        self._tools = {}

    def register(self, tool: ToolDef):
        self._tools[tool.name] = tool

    def get(self, name: str) -> ToolDef:
        if name not in self._tools:
            raise KeyError(f"Unknown tool: {name}")
        return self._tools[name]

    def list(self):
        return list(self._tools.keys())

# ------------build registry------------
registry = ToolRegistry()

registry.register(ToolDef(
    "create_plan",
    create_plan,
    "Create a plan from goal",
    schema={ "type": "object", "properties": {}, "required": [] },   # no kwargs
    returns={
        "type": "object",
        "properties": {
            "message": { "type": "string" },
            "steps":   { "type": "array", "items": { "type": "string" } }
        },
        "required": ["message", "steps"]
    }
))

registry.register(ToolDef(
  "read_txt_file", read_txt_file, "Read a .txt file from input_folder",
  schema={
    "type": "object",
    "properties": {"file_name": {"type": "string"}},
    "required": ["file_name"]
  },
  returns={
    "type": "object",
    "properties": {"message": {"type": "string"}, "length": {"type": "integer"}},
    "required": ["message"]
  },
  # side_effects=("read",), version="1.0"
))

registry.register(ToolDef(
    "list_txt_files",
    list_txt_files,
    "List .txt files in input_folder",
    schema={ "type": "object", "properties": {}, "required": [] },
    returns={
        "type": "object",
        "properties": {
            "ok":    { "type": "boolean" },
            "message": { "type": "string" },
            "files": { "type": "array", "items": { "type": "string" } },
            "count": { "type": "integer" }
        },
        "required": ["ok", "files"]
    }
))

registry.register(ToolDef(
  "generate_summary_prompt", generate_summary_prompt, "Build a summarization prompt",
  schema={
    "type": "object",
    "properties": {"max_len": {"type": "integer", "minimum": 1}},
    "required": []
  }
))

registry.register(ToolDef(
  "summarize", summarize, "Run LLM summarization",
  schema={"type": "object", "properties": {}, "required": []}
))

registry.register(ToolDef(
  "save_summary", save_summary, "Persist summary to output_folder",
  schema={
    "type": "object",
    "properties": {"out_name": {"type": "string"}},
    "required": []
  },
  # side_effects=("write",)
))

#------------Environment--------------
def _validate(schema, kwargs):
    if not schema:
        return None
    missing = [k for k in schema.get("required", []) if k not in kwargs]
    if missing:
        return f"Missing required: {missing}"
    types = {"string": str, "integer": int, "number": (int, float), "boolean": bool}
    for k, spec in (schema.get("properties") or {}).items():
        if k in kwargs and "type" in spec:
            py_t = types.get(spec["type"])
            if py_t and not isinstance(kwargs[k], py_t):
                return f"Bad type for '{k}': expected {spec['type']}"
    return None

class Environment:
    def __init__(self, ctx: ActionContext, registry: ToolRegistry):
        self.ctx = ctx
        self.registry = registry

    def run(self, tool_name: str, **kwargs):
        tool = self.registry.get(tool_name)
        fn = tool.func
        sig = inspect.signature(fn)

        # 1) Schema validation BEFORE logging/exec
        v_err = _validate(tool.schema, kwargs)
        if v_err:
            self.ctx.track_progress(tool.name, "error", note=v_err[:180])
            return err(v_err)  # use standardized envelope

        # 2) Build call args with auto-DI (ctx + underscore deps)
        call_args = {}
        for pname, param in sig.parameters.items():
            if pname == "ctx":
                call_args["ctx"] = self.ctx
            elif pname.startswith("_"):   # underscore dep, e.g. _fs, _clock
                dname = pname[1:]
                if dname not in self.ctx.deps:
                    msg = f"Missing dep '{dname}' for tool '{tool_name}'"
                    self.ctx.track_progress(tool.name, "error", note=msg[:180])
                    return err(msg)
                call_args[pname] = self.ctx.deps[dname]
            else:
                if pname in kwargs:
                    call_args[pname] = kwargs[pname]
                elif param.default is not inspect._empty:
                    pass
                else:
                    msg = f"Missing required arg '{pname}' for tool '{tool_name}'"
                    self.ctx.track_progress(tool.name, "error", note=msg[:180])
                    return err(msg)

        # 3) Log start, call tool
        self.ctx.track_progress(tool.name, "started", note=str(kwargs))
        try:
            result = fn(**call_args)
        except Exception as e:
            # Normalize exceptions into err(...) so the agent can handle them
            msg = f"{type(e).__name__}: {e}"
            self.ctx.track_progress(tool.name, "error", note=msg[:180])
            return err(msg)

        # 4) Normalize + log outcome
        if isinstance(result, dict):
            # If tool used envelope:
            if result.get("ok") is False:
                self.ctx.track_progress(tool.name, "error", note=str(result.get("error", ""))[:180])
                return result
            # Back-compat: dict returned with "error" but no "ok"
            if "ok" not in result and "error" in result:
                self.ctx.track_progress(tool.name, "error", note=str(result["error"])[:180])
                return {"ok": False, **result}
            # Success path: ensure ok=True for consistency
            result = result if "ok" in result else {"ok": True, **result}
            note = result.get("message", "")[:120]
            self.ctx.track_progress(tool.name, "completed", note=note)
            return result

        # Non-dict success (rare): mark completed with empty note
        self.ctx.track_progress(tool.name, "completed", note="")
        return result


# ---------- Scripted Agent ----------
class ScriptedAgent:
    def __init__(self, env, steps):
        self.env = env
        self.steps = steps

    def run(self, max_calls=None, stop_on_error=True):
        calls = 0
        for name, kwargs in self.steps:
            if max_calls is not None and calls >= max_calls:
                return {"final": f"stopped: max_calls={max_calls}"}
            res = self.env.run(name, **(kwargs or {}))
            calls += 1
            if stop_on_error and isinstance(res, dict) and res.get("ok") is False:
                # include hint so you know the next best step
                out = {"final": f"stopped at {name}: {res['error']}"}
                if "hint" in res: out["hint"] = res["hint"]
                return out
        return {"final": "done"}

# ---------------- Setup ----------------
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")

config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output",
    # "summary_max_chars": 2400,  # optional
}

llm = OpenAILLM(
    client,
    model=config.get("model", "gpt-4o-mini"),
    temperature=config.get("temperature", 0.2),
)

# Create context with DI bag pre-populated
ctx = ActionContext(memory=memory, llm=llm, config=config, deps={"fs": RealFS})

# Ensure folders exist
os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

# ------------Build env ------------#
env = Environment(ctx, registry)

# --- define the steps your agent should run ---
file_name = "004_AGENT_Tools.txt"
steps = [
  ("create_plan", {}),
  ("read_txt_file", {"file_name": "does_not_exist.txt"}),
  ("generate_summary_prompt", {}),
  ("summarize", {}),
  ("save_summary", {}),
]
final = ScriptedAgent(env, steps).run()
print("Agent result:", final["final"])
print("💡 Hint:", final.get("hint",""))
print(env.run("list_txt_files"))

final = agent.run(max_calls=10)  # optional guard
print("Agent result:", final["final"])
if "hint" in final:
    print("💡 Hint:", final["hint"])

# --- pretty prints from memory (no second calls) ---
plan = ctx.memory.get("plan") or []
print("\nPlan:")
for s in plan:
    print("-", s)

raw_text = ctx.memory.get("raw_text") or ""
print("\n📄 File Preview:\n")
print(textwrap.fill(raw_text[:600], width=80, subsequent_indent="  "))

prompt = ctx.memory.get("summary_prompt") or ""
print("\n🧾 Prompt Preview:\n")
print(textwrap.fill(prompt[:600], width=80, subsequent_indent="  "))

summary = ctx.memory.get("summary") or ""
print("\n📝 Summary Preview:\n")
print(textwrap.fill(summary[:1000], width=80, subsequent_indent="  "))

if ctx.memory.get("summary_path"):
    print("\n📄 Saved to:", ctx.memory.get("summary_path"))

print("\n" + "="*80)
print("📦 ActionContext Snapshot")
ctx.print_progress()



Agent result: stopped at read_txt_file: File not found: /content/files/does_not_exist.txt
💡 Hint: Call list_txt_files to see available files
{'ok': True, 'message': 'Found 3 .txt files.', 'files': ['003_Agent Feedback and Memory.txt', '004_AGENT_Tools.txt', '005_Using Function Calling Capabilities with LLMs.txt'], 'count': 3}
Agent result: done

Plan:
- Open the text file using a text editor or programming tool
- Read the entire content of the file
- Identify the main ideas and key points in the text
- Take notes on important details and supporting information
- Organize the notes into a coherent structure
- Write a concise summary based on the organized notes
- Review and edit the summary for clarity and brevity

📄 File Preview:



🧾 Prompt Preview:



📝 Summary Preview:



📦 ActionContext Snapshot

📊 Progress Log:
- [completed] setup (2025-08-26 23:12:14) — goal + config injected
- [started] create_plan (2025-08-26 23:12:14) — {}
- [completed] create_plan (2025-08-26 23:12:16) — Plan

In [ ]:
# --- define the steps your agent should run ---
file_name = "004_AGENT_Tools.txt"  # from list_txt_files
steps = [
    ("create_plan", {}),
    ("read_txt_file", {"file_name": file_name}),
    ("generate_summary_prompt", {}),
    ("summarize", {}),
    ("save_summary", {}),
]
final = ScriptedAgent(env, steps).run()
print("Agent result:", final["final"])
if "hint" in final: print("💡 Hint:", final["hint"])


# --- pretty prints from memory (no second calls) ---
plan = ctx.memory.get("plan") or []
print("\nPlan:")
for s in plan:
    print("-", s)

raw_text = ctx.memory.get("raw_text") or ""
print("\n📄 File Preview:\n")
print(textwrap.fill(raw_text[:600], width=80, subsequent_indent="  "))

prompt = ctx.memory.get("summary_prompt") or ""
print("\n🧾 Prompt Preview:\n")
print(textwrap.fill(prompt[:600], width=80, subsequent_indent="  "))

summary = ctx.memory.get("summary") or ""
print("\n📝 Summary Preview:\n")
print(textwrap.fill(summary[:1000], width=80, subsequent_indent="  "))

if ctx.memory.get("summary_path"):
    print("\n📄 Saved to:", ctx.memory.get("summary_path"))

print("\n" + "="*80)
print("📦 ActionContext Snapshot")
ctx.print_progress()


Agent result: done

Plan:
- Open the text file using a text editor or programming tool
- Read the entire content of the file
- Identify the main ideas and key points in the text
- Take notes on important details and supporting information
- Organize the notes into a coherent structure (e.g., introduction, body, conclusion)
- Write a concise summary based on the organized notes
- Review and edit the summary for clarity and brevity
- Save the summary in a desired format or location

📄 File Preview:

 #===========AI-Agent Tool Descriptions and Naming  Describing Tools to the
  Agent  When developing an agentic AI system, one of the most critical aspects
  is ensuring that the agent understands the tools it has access to. In our
  previous tutorial, we explored how an AI agent interacts with an environment.
  Now, we extend that discussion to focus on tool definition, particularly the
  importance of naming, parameters, and structured metadata.  Example:
  Automating Documentation for Pyth

In [ ]:
print("\n📄 File Preview:\n", (ctx.memory.get("raw_text") or "")[:600])
print("\n🧾 Prompt Preview:\n", (ctx.memory.get("summary_prompt") or "")[:600])
print("\n📝 Summary Preview:\n", (ctx.memory.get("summary") or "")[:600])
print("\n📄 Saved to:", ctx.memory.get("summary_path"))
ctx.print_progress()



📄 File Preview:
 
#===========AI-Agent Tool Descriptions and Naming

Describing Tools to the Agent

When developing an agentic AI system, one of the most critical aspects is ensuring that the agent understands the tools it has access to. In our previous tutorial, we explored how an AI agent interacts with an environment. Now, we extend that discussion to focus on tool definition, particularly the importance of naming, parameters, and structured metadata.

Example: Automating Documentation for Python Code
Imagine we are building an AI agent that scans through all Python files in a src/ directory and automatical

🧾 Prompt Preview:
 You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points...
"""
#===========AI-Agent Tool Descriptions and Naming

Describing Tools to the Agent

When developing an agentic AI system, one of the most critical aspects is ensuring that the agent understands the tools it has access to. In our previous tutori

This is looking great—the pipeline ran end-to-end, saved the file, and your JIT guidance flow worked exactly as designed. The empty previews in the first run were expected (the agent stopped at `read_txt_file`), and the second run with a valid filename produced previews + a saved summary. ✅

Here’s where you stand vs. the gaps:

* **#1 Standard envelope** — **Done.** Tools return `ok(...)` / `err(...)`, and `Environment.run` handles them.
* **#2 JIT error guidance** — **Done.** `read_txt_file` emits a `hint`, and you implemented `list_txt_files`; flow works (your results show it).
* **#3 Path safety** — **Done (core).** You guard against traversal; optional size cap still a nice-to-have.
* **#4 Fake/in-memory env for tests** — **Partial.** You added underscore-DI and `RealFS`. If you want this fully closed, add a tiny fake FS for tests.

If you want a super-light **FakeFS** for unit tests, here’s a minimal drop-in you can use with your underscore DI:

```python
import io, posixpath
from contextlib import contextmanager

class FakeFS:
    path = posixpath
    def __init__(self):
        self._files = {}  # path -> str
    @staticmethod
    def makedirs(path, exist_ok=True):  # no-op
        return
    @contextmanager
    def open(self, path, mode="r", encoding=None):
        if "w" in mode:
            buf = io.StringIO()
            try:
                yield buf
            finally:
                self._files[path] = buf.getvalue()
        else:
            data = self._files.get(path, "")
            yield io.StringIO(data)
```

Use it in tests:

```python
ctx.deps["fs"] = FakeFS()
# run agent...
# assert ctx.deps["fs"]._files[ctx.memory["summary_path"]] contains expected text
```

Tiny polish you can add anytime:

* Optional **size guard** in `read_txt_file` (e.g., `max_read_bytes` in config).
* Add `returns` metadata to the last couple of tools in the registry (just for documentation/planner readiness) Otherwise, you’re in excellent shape.


# Final Code


In [11]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ SETUP (Notebook-only)                                                        ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
!pip -q install openai python-dotenv


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ IMPORTS                                                                      ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap
import time
import re
import inspect
from typing import Callable, Optional
from dataclasses import dataclass
import builtins


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ OPENAI CLIENT & ENV VARS                                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
# Loads API key from a .env file and initializes the OpenAI client.
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ STANDARD RESULT ENVELOPE (ok / err)                                          ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
def ok(**data):
    """Successful tool result. Add any fields you like."""
    return {"ok": True, **data}

def err(msg, hint=None, retryable=False, **extra):
    """Error result with optional guidance and flags."""
    out = {"ok": False, "error": msg, "retryable": retryable}
    if hint:
        out["hint"] = hint
    if extra:
        out.update(extra)
    return out


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ FILESYSTEM ADAPTER (for underscore-DI: _fs)                                  ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
# RealFS exposes .path/.makedirs/.open so tools can accept a pluggable FS.
class RealFS:
    path = os.path
    makedirs = staticmethod(os.makedirs)
    open = staticmethod(builtins.open)


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ MEMORY & CONTEXT                                                             ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
class ScratchMemory:
    """Minimal in-memory key/value store for agent state."""
    def __init__(self):
        self.store = {}

    def get(self, key, default=None):   # default added for convenience
        return self.store.get(key, default)

    def set(self, key, value):
        self.store[key] = value

# Valid progress states for centralized logging.
VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    """
    The agent's 'backpack':
      - memory: state across steps
      - llm:    LLM wrapper
      - config: runtime configuration (folders, knobs)
      - deps:   injectable dependencies (e.g., fs/clock)
    """
    def __init__(self, memory, llm, config=None, deps=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}
        self.deps = deps or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None

# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ LLM WRAPPER                                                                 ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini", temperature=0.2):
        self.client = client
        self.model = model
        self.temperature = temperature

    def complete(self, prompt, **kwargs):
        temp = kwargs.get("temperature", self.temperature)
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
        )
        return resp.choices[0].message.content


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ TOOLS: PLANNING                                                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return err("No goal provided (memory key 'goal' missing).",
                   hint="Set ctx.memory['goal'] before calling create_plan")

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond ONLY with a numbered list, one step per line. No extra prose."""
    raw = ctx.llm.complete(prompt).strip()

    # Prefer numbered steps like "1. ...", "2) ..."
    numbered = re.findall(r'^\s*(?:\d+[\).\s-]+)\s*(.+)$', raw, flags=re.M)

    if numbered:
        steps = numbered
    else:
        # Fallback to bullets like "- ...", "* ...", "• ..."
        bullets = re.findall(r'^\s*(?:[-*•]\s+)(.+)$', raw, flags=re.M)
        steps = bullets if bullets else [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # Normalize: collapse spaces, trim punctuation, drop empties/dupes
    clean_steps = []
    seen = set()
    for s in steps:
        s = re.sub(r'\s+', ' ', s).strip(' .')
        if s and s.lower() not in seen:
            seen.add(s.lower())
            clean_steps.append(s)

    if not clean_steps:
        return err("Planner returned no steps.",
                   hint="Refine the goal or relax the parser constraints")

    ctx.memory.set("plan", clean_steps)
    # optional: match handbook wording
    # ctx.memory.set("current_plan", clean_steps)

    return ok(message="Plan created from goal.", steps=clean_steps)


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ TOOLS: I/O (FILES)                                                           ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
def read_txt_file(ctx, file_name):
    base = os.path.abspath(ctx.config.get("input_folder", ""))
    path = os.path.abspath(os.path.join(base, file_name))
    if not base or not path.startswith(base + os.sep):
        return err("Path traversal blocked.", retryable=False)

    if not os.path.exists(path):
        return err(f"File not found: {path}",
                   hint="Call list_txt_files to see available files",
                   retryable=True)

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("file_name", file_name)
    ctx.memory.set("raw_text", text)
    return ok(message="File read successfully.", length=len(text))


# ── Helper: list available .txt files (for JIT guidance) ───────────────────────
def list_txt_files(ctx):
    base = ctx.config.get("input_folder")
    if not base:
        return err("No input_folder in config.", hint="Set ctx.config['input_folder']")
    if not os.path.isdir(base):
        return err(f"Input folder not found: {base}", retryable=False)

    files = sorted(f for f in os.listdir(base) if f.endswith(".txt"))
    ctx.memory.set("available_txt_files", files)  # optional: stash for UI/agent
    return ok(message=f"Found {len(files)} .txt files.", files=files, count=len(files))


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ TOOLS: SUMMARIZATION                                                         ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
def generate_summary_prompt(ctx, max_len=None):
    text = ctx.memory.get("raw_text")
    if not text:
        return err("No raw text found in memory.",
                   hint="Run read_txt_file before generate_summary_prompt")
    if max_len is None:
        max_len = ctx.config.get("summary_max_chars", 2000)

    truncated = len(text) > max_len
    short_text = text[:max_len]

    ctx.memory.set("was_truncated", truncated)
    ctx.memory.set("source_length", len(text))
    ctx.memory.set("used_length", len(short_text))
    ctx.memory.set("summary_prompt", f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points...
\"\"\"{short_text}\"\"\"

Summary:""")

    return ok(message="Summary prompt created.",
              truncated=truncated, used=len(short_text), total=len(text),
              prompt_preview=ctx.memory.get("summary_prompt")[:600])


def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return err("No summary prompt found in memory.",
                   hint="Run generate_summary_prompt before summarize")
    response = ctx.llm.complete(prompt)
    ctx.memory.set("summary", response)
    return ok(message="Summary completed.", summary_preview=response[:1000])


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ TOOLS: OUTPUT                                                                ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
def save_summary(ctx, out_name=None, _fs=os):
    summary = ctx.memory.get("summary")
    if not summary:
        return err("No summary in memory.",
                   hint="Run summarize before save_summary")
    out_dir = ctx.config.get("output_folder")
    if not out_dir:
        return err("No output_folder in config.",
                   hint="Set ctx.config['output_folder']")

    _fs.makedirs(out_dir, exist_ok=True)
    src = ctx.memory.get("file_name", "summary")
    root, _ = os.path.splitext(os.path.basename(src))
    base = out_name or f"{root}_summary.txt"
    path = _fs.path.join(out_dir, base)

    with _fs.open(path, "w", encoding="utf-8") as f:
        f.write(summary)

    ctx.memory.set("summary_path", path)
    return ok(message="Summary saved.", path=path)

# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ TOOL REGISTRY — TYPES & REGISTRATION                                         ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
from dataclasses import dataclass
from typing import Callable

@dataclass
class ToolDef:
    name: str
    func: Callable
    description: str = ""
    schema: dict | None = None
    returns: dict | None = None   # optional metadata about outputs

class ToolRegistry:
    def __init__(self):
        self._tools = {}

    def register(self, tool: ToolDef):
        self._tools[tool.name] = tool

    def get(self, name: str) -> ToolDef:
        if name not in self._tools:
            raise KeyError(f"Unknown tool: {name}")
        return self._tools[name]

    def list(self):
        return list(self._tools.keys())

# -- Build registry -------------------------------------------------------------
registry = ToolRegistry()

registry.register(ToolDef(
    "create_plan",
    create_plan,
    "Create a plan from goal",
    schema={ "type": "object", "properties": {}, "required": [] },   # no kwargs
    returns={
        "type": "object",
        "properties": {
            "message": { "type": "string" },
            "steps":   { "type": "array", "items": { "type": "string" } }
        },
        "required": ["message", "steps"]
    }
))

registry.register(ToolDef(
  "read_txt_file", read_txt_file, "Read a .txt file from input_folder",
  schema={
    "type": "object",
    "properties": {"file_name": {"type": "string"}},
    "required": ["file_name"]
  },
  returns={
    "type": "object",
    "properties": {"message": {"type": "string"}, "length": {"type": "integer"}},
    "required": ["message"]
  },
))

registry.register(ToolDef(
    "list_txt_files",
    list_txt_files,
    "List .txt files in input_folder",
    schema={ "type": "object", "properties": {}, "required": [] },
    returns={
        "type": "object",
        "properties": {
            "ok":    { "type": "boolean" },
            "message": { "type": "string" },
            "files": { "type": "array", "items": { "type": "string" } },
            "count": { "type": "integer" }
        },
        "required": ["ok", "files"]
    }
))

registry.register(ToolDef(
  "generate_summary_prompt", generate_summary_prompt, "Build a summarization prompt",
  schema={
    "type": "object",
    "properties": {"max_len": {"type": "integer", "minimum": 1}},
    "required": []
  }
))

registry.register(ToolDef(
  "summarize", summarize, "Run LLM summarization",
  schema={"type": "object", "properties": {}, "required": []}
))

registry.register(ToolDef(
  "save_summary", save_summary, "Persist summary to output_folder",
  schema={
    "type": "object",
    "properties": {"out_name": {"type": "string"}},
    "required": []
  },
))


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ ENVIRONMENT — VALIDATION & EXECUTION                                         ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
import inspect

def _validate(schema, kwargs):
    """Minimal JSON-schema-ish validator for tool kwargs."""
    if not schema:
        return None
    missing = [k for k in schema.get("required", []) if k not in kwargs]
    if missing:
        return f"Missing required: {missing}"
    types = {"string": str, "integer": int, "number": (int, float), "boolean": bool}
    for k, spec in (schema.get("properties") or {}).items():
        if k in kwargs and "type" in spec:
            py_t = types.get(spec["type"])
            if py_t and not isinstance(kwargs[k], py_t):
                return f"Bad type for '{k}': expected {spec['type']}"
    return None

class Environment:
    """Runs tools by name with auto-DI, validation, and centralized logging."""
    def __init__(self, ctx: ActionContext, registry: ToolRegistry):
        self.ctx = ctx
        self.registry = registry

    def run(self, tool_name: str, **kwargs):
        tool = self.registry.get(tool_name)
        fn = tool.func
        sig = inspect.signature(fn)

        # 1) Schema validation BEFORE logging/exec
        v_err = _validate(tool.schema, kwargs)
        if v_err:
            self.ctx.track_progress(tool.name, "error", note=v_err[:180])
            return err(v_err)  # standardized envelope

        # 2) Build call args with auto-DI (ctx + underscore deps)
        call_args = {}
        for pname, param in sig.parameters.items():
            if pname == "ctx":
                call_args["ctx"] = self.ctx
            elif pname.startswith("_"):   # underscore dep, e.g. _fs, _clock
                dname = pname[1:]
                if dname not in self.ctx.deps:
                    msg = f"Missing dep '{dname}' for tool '{tool_name}'"
                    self.ctx.track_progress(tool.name, "error", note=msg[:180])
                    return err(msg)
                call_args[pname] = self.ctx.deps[dname]
            else:
                if pname in kwargs:
                    call_args[pname] = kwargs[pname]
                elif param.default is not inspect._empty:
                    pass
                else:
                    msg = f"Missing required arg '{pname}' for tool '{tool_name}'"
                    self.ctx.track_progress(tool.name, "error", note=msg[:180])
                    return err(msg)

        # 3) Log start, call tool
        self.ctx.track_progress(tool.name, "started", note=str(kwargs))
        try:
            result = fn(**call_args)
        except Exception as e:
            # Normalize exceptions into err(...) so the agent can handle them
            msg = f"{type(e).__name__}: {e}"
            self.ctx.track_progress(tool.name, "error", note=msg[:180])
            return err(msg)

        # 4) Normalize + log outcome
        if isinstance(result, dict):
            # If tool used envelope:
            if result.get("ok") is False:
                self.ctx.track_progress(tool.name, "error", note=str(result.get("error", ""))[:180])
                return result
            # Back-compat: dict returned with "error" but no "ok"
            if "ok" not in result and "error" in result:
                self.ctx.track_progress(tool.name, "error", note=str(result["error"])[:180])
                return {"ok": False, **result}
            # Success path: ensure ok=True for consistency
            result = result if "ok" in result else {"ok": True, **result}
            note = result.get("message", "")[:120]
            self.ctx.track_progress(tool.name, "completed", note=note)
            return result

        # Non-dict success (rare): mark completed with empty note
        self.ctx.track_progress(tool.name, "completed", note="")
        return result


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ SCRIPTED AGENT — FIXED PIPELINE RUNNER                                       ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
class ScriptedAgent:
    """Executes a predetermined sequence of (tool_name, kwargs) steps."""
    def __init__(self, env, steps):
        self.env = env
        self.steps = steps

    def run(self, max_calls=None, stop_on_error=True):
        calls = 0
        for name, kwargs in self.steps:
            if max_calls is not None and calls >= max_calls:
                return {"final": f"stopped: max_calls={max_calls}"}
            res = self.env.run(name, **(kwargs or {}))
            calls += 1
            if stop_on_error and isinstance(res, dict) and res.get("ok") is False:
                # include hint so you know the next best step
                out = {"final": f"stopped at {name}: {res['error']}"}
                if "hint" in res: out["hint"] = res["hint"]
                return out
        return {"final": "done"}


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ SETUP & CONFIG                                                               ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")

config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output",
    # "summary_max_chars": 2400,  # optional
}

llm = OpenAILLM(
    client,
    model=config.get("model", "gpt-4o-mini"),
    temperature=config.get("temperature", 0.2),
)


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ CONTEXT & ENVIRONMENT                                                        ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
# Create context with DI bag pre-populated (fs adapter)
ctx = ActionContext(memory=memory, llm=llm, config=config, deps={"fs": RealFS})

# Ensure folders exist (lightweight guardrails)
os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

# Build environment (validation + auto-DI + centralized logging)
env = Environment(ctx, registry)


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ AGENT STEPS (SCRIPTED PIPELINE)                                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
file_name = "004_AGENT_Tools.txt"
steps = [
    ("create_plan", {}),
    ("read_txt_file", {"file_name": file_name}),
    ("generate_summary_prompt", {}),  # or {"max_len": 2400}
    ("summarize", {}),
    ("save_summary", {}),
]


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ RUN AGENT                                                                    ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
agent = ScriptedAgent(env, steps)
final = agent.run(max_calls=10)  # optional guard
print("Agent result:", final["final"])
if "hint" in final:
    print("💡 Hint:", final["hint"])


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ PRETTY PRINTS (FROM MEMORY)                                                  ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
plan = ctx.memory.get("plan") or []
print("\nPlan:")
for s in plan:
    print("-", s)

raw_text = ctx.memory.get("raw_text") or ""
print("\n📄 File Preview:\n")
print(textwrap.fill(raw_text[:600], width=80, subsequent_indent="  "))

prompt = ctx.memory.get("summary_prompt") or ""
print("\n🧾 Prompt Preview:\n")
print(textwrap.fill(prompt[:600], width=80, subsequent_indent="  "))

summary = ctx.memory.get("summary") or ""
print("\n📝 Summary Preview:\n")
print(textwrap.fill(summary[:1000], width=80, subsequent_indent="  "))

if ctx.memory.get("summary_path"):
    print("\n📄 Saved to:", ctx.memory.get("summary_path"))


# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║ CONTEXT SNAPSHOT / PROGRESS LOG                                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝
print("\n" + "="*80)
print("📦 ActionContext Snapshot")
ctx.print_progress()


Agent result: done

Plan:
- Open the text file using a text editor or programming tool
- Read the entire content of the file
- Identify the main ideas and key points in the text
- Take notes on important details and supporting information
- Organize the notes into a coherent structure
- Write a concise summary based on the organized notes
- Review and edit the summary for clarity and conciseness
- Save the summary in a desired format or location

📄 File Preview:

 #===========AI-Agent Tool Descriptions and Naming  Describing Tools to the
  Agent  When developing an agentic AI system, one of the most critical aspects
  is ensuring that the agent understands the tools it has access to. In our
  previous tutorial, we explored how an AI agent interacts with an environment.
  Now, we extend that discussion to focus on tool definition, particularly the
  importance of naming, parameters, and structured metadata.  Example:
  Automating Documentation for Python Code Imagine we are building an 